# Test Vari

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import sys 
sys.path.append("../")
import torch
from monai.transforms import (
    Activations,
    AsChannelFirst,
    AsDiscrete,
    Compose,
    RandFlip,
    RandRotate,
    RandZoom,
    Resize
)
# from torch.utils.data import DataLoader
from monai.data import DataLoader
from monai.networks.nets import DenseNet121
from sklearn.model_selection import train_test_split

In [146]:
T1  = ["/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/T1/" + x for x in os.listdir("/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/T1/")]
T2  = ["/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/T2/" + x for x in os.listdir("/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/T2/")]
FLAIR  = ["/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/FLAIR/" + x for x in os.listdir("/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/FLAIR/")]
OTHER  = ["/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/OTHER/" + x for x in os.listdir("/home/jovyan/Verdicchio/ONWAY_DATA/Dataset/OTHER/")]


X = T1 + T2 + FLAIR +OTHER 
y = list(np.zeros(len(T1),dtype = int)) + list(np.ones(len(T2),dtype = int)) + list(np.ones(len(FLAIR),dtype = int)) + list(3*np.ones(len(OTHER),dtype = int))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True ,random_state=42,stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2,shuffle=True ,random_state=42,stratify=y_train)

print("Le dimensioni del dataset sono : ")
print(f"Train {len(X_train)}")
print(f"Validation {len(X_val)}")
print(f"Test {len(X_test)}")


Le dimensioni del dataset sono : 
Train 1132
Validation 283
Test 354


In [77]:
class Sequence_Dataset(torch.utils.data.Dataset) : 
    def __init__(self,image_files, transforms): 
        self.image_files = image_files
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_files)
    
    
    def __getitem__(self,index) :
        img = np.load(self.image_files[index])
        label = os.path.basename(os.path.dirname(self.image_files[index]))
        return self.transforms(img), label

In [101]:
train_transforms = Compose(
    [   AsChannelFirst(2),
        Resize((256,256)),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
    ]
)

val_transforms = Compose(
    [   AsChannelFirst(2),
        Resize((256,256)),
            ]
)


In [147]:
train_ds = Sequence_Dataset(image_files=X_train,transforms=train_transforms)
train_loader = DataLoader(train_ds,batch_size  = 50, shuffle=True, num_workers=8 , pin_memory=True)

# Pytorch Lightining

## Dataset Definition 




In [204]:
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
import os
import numpy as np
from monai.transforms import (
    AsChannelFirst,
    Compose,
    RandFlip,
    RandRotate,
    RandZoom,
    Resize
)
from monai.data import DataLoader

def labeling(data) :
    label = []
    for path in data : 
        label.append(os.path.basename(os.path.dirname(path)))
    return label

train_transforms = Compose(
    [   AsChannelFirst(2),
        Resize((256,256)),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
    ]
)

val_transforms = Compose(
    [   AsChannelFirst(2),
        Resize((256,256)),
            ]
)
    

class Sequence_Dataset(torch.utils.data.Dataset) : 
    def __init__(self,image_files,label, transforms): 
        self.image_files = image_files
        self.transforms = transforms
        self.label = label
    def __len__(self):
        return len(self.image_files)
    
    
    def __getitem__(self,index) :
        img = np.load(self.image_files[index])
        return self.transforms(img), self.label[index]

class DataModule(pl.LightningDataModule) : 
    
    def __init__(self, data_dir : list , batch_size : int ) : 
        
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.__ = None
        
    def prepare_data(self) :
        y = labeling(X)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2,shuffle=True ,random_state=42,stratify=y)
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(X_train, y_train, test_size=0.2,shuffle=True ,random_state=42,stratify=y_train)
    
    def setup(self,stage : str) : 
        
        if stage == "fit" : 
            self.train_ds =  Sequence_Dataset(self.X_train,self.y_train,train_transforms)
            self.val_ds = Sequence_Dataset(self.X_val,self.y_val,val_transforms)
            
        if stage == "test" : 
            self.test_ds = Sequence_Dataset(self.X_test,self.y_test,val_transforms)
            
    def train_dataloader(self) :
        
        return DataLoader(self.train_ds, batch_size  = self.batch_size, shuffle=True, num_workers=8 , pin_memory=True)

    def val_dataloader(self) :
        
        return DataLoader(self.val_ds, batch_size  = self.batch_size, shuffle=True, num_workers=8 , pin_memory=True)

    def test_dataloader(self) :
        
        return DataLoader(self.test_ds, batch_size  = self.batch_size, shuffle=True, num_workers=8 , pin_memory=True)



/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AsChannelFirst'>: Class `AsChannelFirst` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


In [201]:
dm = DataModule(X,1)
dm.prepare_data()
dm.setup(stage ="fit")
A = next(iter(dm.train_dataloader()))

## Model Definition

In [205]:
import pytorch_lightning as pl
from monai.networks.nets import ResNet




class Model(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def forward(self, x) : 
        self.model(x)
        
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self._model.parameters(), lr=1e-4, weight_decay=1e-5)
        return optimizer
